In [1]:
import torch
import cmw_utils

In [2]:
x_param = torch.tensor([2.0, 2.0], requires_grad=True)
y_param = torch.tensor([2.0, 2.0], requires_grad=True)

# x_loss = torch.dot(x_param, x_param) * torch.dot(y_param, y_param)
# y_loss =  - torch.dot(x_param, x_param) * torch.dot(y_param, y_param)

x_loss = torch.sum(torch.pow(x_param, 2)) * torch.sum(torch.pow(y_param, 2))
y_loss = - torch.sum(torch.pow(x_param, 2)) * torch.sum(torch.pow(y_param, 2))

b1 = torch.tensor([1.0, 1.0])
b2 = torch.tensor([1.0, 1.0])


result1 = cmw_utils.avp([x_loss, y_loss], [x_param, y_param], [b1, b2], transpose=False, retain_graph=True)
result2 = cmw_utils.avp([x_loss, y_loss], [x_param, y_param], [b1, b2], transpose=True, retain_graph=True)

print(result1)
print(result2)


[tensor([32.5000, 32.5000], grad_fn=<AddBackward0>), tensor([-31.5000, -31.5000], grad_fn=<AddBackward0>)]
[tensor([-31.5000, -31.5000], grad_fn=<AddBackward0>), tensor([32.5000, 32.5000], grad_fn=<AddBackward0>)]


In [2]:
x_param = torch.tensor([1.0], requires_grad=True)
y_param = torch.tensor([1.0], requires_grad=True)

alpha = 1.

x_loss = alpha * (torch.sum(x_param) - 0.1) * (torch.sum(y_param) - 0.1)
y_loss = - alpha * (torch.sum(x_param) - 0.1) * (torch.sum(y_param) - 0.1)

nash_list, n_iter = cmw_utils.metamatrix_conjugate_gradient(
    [x_loss, y_loss], [x_param, y_param])

new_params = cmw_utils.project_update(nash_list, [x_param, y_param])
print(new_params)

[tensor([0.4066], requires_grad=True), tensor([1.], requires_grad=True)]


In [4]:
x_param = torch.tensor([0.2], requires_grad=True)
y_param = torch.tensor([0.2], requires_grad=True)

alpha = 0.1
num_iter = 100

param_list = [x_param, y_param]

for n in range(num_iter):
    print(param_list)
    
    x_loss = alpha * (torch.sum(param_list[0]) - 0.1) * (torch.sum(param_list[1]) - 0.1)
    y_loss = - alpha * (torch.sum(param_list[0]) - 0.1) * (torch.sum(param_list[1]) - 0.1)
    
    nash_list, n_iter = cmw_utils.metamatrix_conjugate_gradient(
        [x_loss, y_loss], param_list)

    param_list = cmw_utils.project_update(nash_list, param_list)
    
print(param_list)

[tensor([0.2000], requires_grad=True), tensor([0.2000], requires_grad=True)]
[tensor([0.1980], requires_grad=True), tensor([0.2020], requires_grad=True)]
[tensor([0.1959], requires_grad=True), tensor([0.2039], requires_grad=True)]
[tensor([0.1939], requires_grad=True), tensor([0.2058], requires_grad=True)]
[tensor([0.1918], requires_grad=True), tensor([0.2077], requires_grad=True)]
[tensor([0.1897], requires_grad=True), tensor([0.2096], requires_grad=True)]
[tensor([0.1876], requires_grad=True), tensor([0.2115], requires_grad=True)]
[tensor([0.1855], requires_grad=True), tensor([0.2133], requires_grad=True)]
[tensor([0.1834], requires_grad=True), tensor([0.2151], requires_grad=True)]
[tensor([0.1812], requires_grad=True), tensor([0.2168], requires_grad=True)]
[tensor([0.1791], requires_grad=True), tensor([0.2185], requires_grad=True)]
[tensor([0.1770], requires_grad=True), tensor([0.2202], requires_grad=True)]
[tensor([0.1748], requires_grad=True), tensor([0.2219], requires_grad=True)]